<a href="https://colab.research.google.com/github/Dixit6054/MY_project/blob/master/Telegram%20bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#inital code, was able to send messages

import flask
import requests

def alert_api(request):
    """
    Sample call:
    https://asia-east2-viet-grade.cloudfunctions.net/alert_api?message=TestMessage
    """
    msg = request.args.get('message')  # Fetch the 'message' parameter from the request
    __send_message(msg)  # Send the message via Telegram
    return flask.jsonify({"status": "done"})  # Return response to the client

def __send_message(msg):
    """
    Send message via Telegram bot
    :param msg: Message to be sent
    :return: Response content from Telegram API
    """
    TELEGRAM_TOKEN = '6752612939:AAHG52PLf3c8XK3Wc-t1ohszKeu8ILx4ISg'  #  Telegram bot token
    TELEGRAM_CHAT_ID = '-1001958631566'  #  Telegram chat ID

    # Payload to send the message via Telegram Bot API
    payload = {
        'chat_id': TELEGRAM_CHAT_ID,
        'text': msg,
        'parse_mode': 'HTML'
    }

    # Send HTTP POST request to Telegram Bot API
    return requests.post(f'https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage',
                         data=payload).content

__send_message('msg')

b'{"ok":true,"result":{"message_id":232,"sender_chat":{"id":-1001958631566,"title":"GBPUSD","type":"channel"},"chat":{"id":-1001958631566,"title":"GBPUSD","type":"channel"},"date":1734489931,"text":"msg"}}'

In [ ]:
!pip install python-telegram-bot
import flask
import requests
import telegram
from flask import jsonify

def telegram_bot(request):
    bot = telegram.Bot(token='6752612939:AAHG52PLf3c8XK3Wc-t1ohszKeu8ILx4ISg')
    if request.method == "POST":
        try:
            # Parse incoming Telegram payload
            update = telegram.Update.de_json(request.get_json(force=True), bot)
            chat_id = update.message.chat.id
            user_message = update.message.text

            # Log incoming data
            print(f"Incoming message: {user_message} from chat ID: {chat_id}")

            # Reply to the sender
            bot.sendMessage(chat_id=chat_id, text="Noted: {user_message}")

            return jsonify({"status": "success"}), 200  # Explicit success response

        except Exception as e:
            print(f"Error: {str(e)}")
            return jsonify({"status": "error", "message": str(e)}), 500

    return jsonify({"status": "error", "message": "Invalid request method"}), 405


In [ ]:
import os
import telegram
from flask import Flask, request, jsonify

app = Flask(__name__)

# Replace with your bot token (hardcoded for testing)
TELEGRAM_TOKEN = "6752612939:AAHG52PLf3c8XK3Wc-t1ohszKeu8ILx4ISg"
bot = telegram.Bot(token=TELEGRAM_TOKEN)

@app.route('/telegram_bot', methods=['POST'])
def telegram_bot():
    try:
        # Parse incoming JSON payload
        update = telegram.Update.de_json(request.get_json(force=True), bot)
        chat_id = update.message.chat.id
        user_message = update.message.text

        print(f"Received message: {user_message} from chat ID: {chat_id}")

        # Reply to the sender
        bot.sendMessage(chat_id=chat_id, text=f"Noted: {user_message}")

        return jsonify({"status": "success", "message": "Message processed"}), 200

    except Exception as e:
        print(f"Error: {str(e)}")
        return jsonify({"status": "error", "message": str(e)}), 400

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [2]:
import os
import telegram
from flask import jsonify, request

# Initialize Telegram Bot with token from environment variables
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
if not TELEGRAM_BOT_TOKEN:
    raise ValueError("TELEGRAM_BOT_TOKEN is not set. Please configure it.")

bot = telegram.Bot(token=TELEGRAM_BOT_TOKEN)

def telegram_bot(request):
    """
    Google Cloud Function to handle Telegram webhook and other HTTP requests.
    Supports POST for Telegram updates and GET for health checks.
    """
    if request.method == "POST":
        try:
            # Parse incoming JSON payload
            if not request.is_json:
                return jsonify({"status": "error", "message": "Request payload is not valid JSON"}), 400

            request_json = request.get_json()
            if not request_json:
                return jsonify({"status": "error", "message": "Request payload is empty"}), 400

            # Parse the Telegram update
            update = telegram.Update.de_json(request_json, bot)

            # Validate update structure
            if not update.message or not update.message.chat or not update.message.text:
                return jsonify({
                    "status": "error",
                    "message": "Invalid Telegram update: Missing 'message', 'chat', or 'text' fields"
                }), 400

            # Extract message details
            chat_id = update.message.chat.id
            user_message = update.message.text

            # Log the received message
            print(f"Received message: '{user_message}' from chat ID: {chat_id}")

            # Use blocking version of send_message
            bot.send_message(chat_id=chat_id, text=f"Noted: {user_message}")

            # Return success response
            return jsonify({"status": "success"}), 200

        except telegram.error.TelegramError as te:
            print(f"Telegram API error: {str(te)}")
            return jsonify({"status": "error", "message": "Telegram API error"}), 500
        except Exception as e:
            print(f"Unexpected error: {str(e)}")
            return jsonify({"status": "error", "message": f"Internal server error: {str(e)}"}), 500

    elif request.method == "GET":
        # Example response for GET requests
        return jsonify({"status": "success", "message": "Telegram bot is up and running!"}), 200

    else:
        # Handle unsupported HTTP methods
        return jsonify({"status": "error", "message": f"Method {request.method} not allowed"}), 405


ModuleNotFoundError: No module named 'telegram'

In [8]:
"""
This script creates a Flask web server that acts as a webhook for a Telegram bot.
It uses ngrok to expose the local server to the internet, allowing Telegram to send updates.
When a message is received, the bot replies with a confirmation message.
"""

import os
import telegram
from flask import Flask, request, jsonify
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)

# Replace this with your Telegram bot token
TELEGRAM_TOKEN = "6752612939:AAHG52PLf3c8XK3Wc-t1ohszKeu8ILx4ISg"  # Replace with your actual Telegram bot token
bot = telegram.Bot(token=TELEGRAM_TOKEN)  # Create a bot instance using the token

# --- Webhook Route ---
# This route handles incoming POST requests from Telegram to the '/telegram_bot' endpoint.
# It processes the incoming message and sends a response back to Telegram.
@app.route('/telegram_bot', methods=['POST'])
def telegram_bot():
    """
    Handles incoming Telegram webhook requests.

    Args:
        None (receives data from the request object).

    Returns:
        A JSON response indicating success or error, along with an HTTP status code.
    """
    try:
        # Parse incoming data
        update = telegram.Update.de_json(request.get_json(force=True), bot)  # Parse the incoming JSON payload into a Telegram Update object
        chat_id = update.message.chat.id  # Get the chat ID from the Update object
        user_message = update.message.text  # Get the user's message from the Update object

        print(f"Received: '{user_message}' from Chat ID: {chat_id}")  # Print the received message and chat ID to the console

        # Reply back with a confirmation message
        bot.sendMessage(chat_id=chat_id, text=f"Noted: {user_message}")  # Send a reply message to the user

        return jsonify({"status": "success", "message": "Message processed"}), 200  # Return a success response with HTTP status code 200

    except Exception as e:
        print(f"Error: {e}")  # Print any errors to the console
        return jsonify({"status": "error", "message": str(e)}), 200  # Return an error response with HTTP status code 200 (to prevent Telegram retries)

# Expose Flask App using ngrok
public_url = ngrok.connect(5000).public_url  # Start ngrok and get the public URL
print(f"Public URL: {public_url}")  # Print the public URL to the console

# Run Flask Server
app.run(port=5000)  # Start the Flask server on port 5000

ERROR:pyngrok.process.ngrok:t=2024-12-18T17:00:19+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-12-18T17:00:19+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-12-18T17:00:19+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [6]:
"""
This script defines a webhook for a Telegram bot.
It listens for incoming messages from Telegram,
processes them (in this case, simply echoing the message back),
and sends a response to the user.
The webhook is designed to be deployed as a serverless function
(e.g., on Google Cloud Functions) and is triggered by
HTTP POST requests from Telegram.
"""

import os
import telegram
from flask import jsonify, request

# Initialize Telegram bot
# TELEGRAM_TOKEN = os.environ.get("TELEGRAM_TOKEN")  # Ensure you set this in your environment, uncomment to use env variable for token
bot = telegram.Bot(token='6752612939:AAHG52PLf3c8XK3Wc-t1ohszKeu8ILx4ISg')  # Initialize with your bot token


# --- Function to handle Telegram Webhook ---
# This function is called whenever Telegram sends an update to the bot.
# It's responsible for receiving the update, processing it, and sending a response back to Telegram.
def telegram_webhook(request):
    """
    This function handles incoming Telegram webhook requests.
    It receives updates from Telegram, processes them,
    and sends a response back.

    Args:
        request: The incoming HTTP request object from Flask.

    Returns:
        A JSON response indicating success or error, along with an HTTP status code.
    """
    try:
        # Log the incoming payload
        payload = request.get_json(force=True)  # Extract Telegram request payload as a dictionary. force=True ensures it's parsed as JSON
        print("Incoming Payload:", payload)  # Log the entire payload for debugging purposes

        # Extract required fields
        update = telegram.Update.de_json(payload, bot)  # Create an Update object from the payload using the bot instance
        chat_id = update.message.chat.id  # Get the chat ID of the sender from the Update object
        user_message = update.message.text  # Get the text message sent by the user from the Update object

        # Log message details
        print(f"Message received: {user_message} | Chat ID: {chat_id}")  # Print message and chat ID to console for logging

        # Send a response back to Telegram
        bot.sendMessage(chat_id=chat_id, text=f"Noted: {user_message}")  # Send a confirmation message to the user using the bot instance

        # Return HTTP 200 OK
        return jsonify({"status": "success", "message": "Processed successfully"}), 200  # Return success response with HTTP 200 OK

    except Exception as e:
        # Log the error and respond with HTTP 200 to prevent retries
        print(f"Error occurred: {str(e)}")  # Print the error message to the console for debugging
        return jsonify({"status": "error", "message": str(e)}), 200  # Return error response but with HTTP 200 to prevent Telegram from retrying

# App to reply back telegram messages


In [3]:
!pip install flask pyngrok python-telegram-bot==13.12


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.5.0
    Uninstalling cachetools-5.5.0:
      Successfully uninstalled cachetools-5.5.0


In [ ]:
# This code creates a flask webserver with a public web link, and the telegram_bot is run on the POST requests
# After this next webhook is set for a bot in next cell, but wull have to go to a differnt collab book

import os
import telegram
from flask import Flask, request, jsonify
from pyngrok import ngrok

# Initialize Flask App
app = Flask(__name__)

# Telegram Bot Token
bot = telegram.Bot(token='6752612939:AAHG52PLf3c8XK3Wc-t1ohszKeu8ILx4ISg')

# Log file path
LOG_FILE = "telegram_bot_logs.txt"

# Function to write logs
def log_to_file(message):
    with open(LOG_FILE, "a") as log_file:
        log_file.write(message + "\n")

@app.route('/telegram_bot', methods=['POST'])
def telegram_bot():
    try:
        # Capture incoming request payload
        payload = request.get_json(force=True)
        log_to_file(f"Received Payload: {payload}")  # Log the payload to file

        # Extract details from the payload
        update = telegram.Update.de_json(payload, bot)
        chat_id = update.message.chat.id
        user_message = update.message.text

        # Log message details
        log_to_file(f"Message: '{user_message}' | Chat ID: {chat_id}")

        # Send response back to Telegram
        bot.sendMessage(chat_id=chat_id, text=f"Noted: {user_message}")

        # Return HTTP 200 OK
        log_to_file("Response sent successfully")
        return jsonify({"status": "success", "message": "Webhook received"}), 200

    except Exception as e:
        log_to_file(f"Error: {e}")
        return jsonify({"status": "error", "message": str(e)}), 200

# Ngrok setup
ngrok.set_auth_token("2qNJ3PecUpB7w9xLEexN2O1V4sq_6RrKvHFciqKZFazDRk29J")  # Auth token
public_url = ngrok.connect(5000).public_url
log_to_file(f"Public URL: {public_url}")
print(f"Public URL: {public_url}")

# Run Flask Server
app.run(port=5000)


Public URL: https://b10e-35-245-170-25.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Dec/2024 05:26:54] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2024 05:26:54] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2024 05:27:13] "POST /telegram_bot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2024 05:27:44] "POST /telegram_bot HTTP/1.1" 200 -


In [ ]:
import requests

# Replace with your bot token and the new webhook URL
TELEGRAM_TOKEN = "6752612939:AAHG52PLf3c8XK3Wc-t1ohszKeu8ILx4ISg"
NEW_WEBHOOK_URL = "https://643b-35-245-170-25.ngrok-free.app/telegram_bot"

# Set the new webhook URL
response = requests.post(
    f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/setWebhook",
    json={"url": NEW_WEBHOOK_URL}
)

# Print the response from Telegram
print("Set Webhook Response:", response.json())


Set Webhook Response: {'ok': False, 'error_code': 404, 'description': 'Not Found'}
